The purpose of this code is to determine the necessary fluence of our x-rays, molarity of our sample, and thickness of our jet.
We have the following constraints:
The first pulse should excite 10% of the molecules.  The second pulse should have high enough fluence and the molarity should be high enough with small enough thickness to get sufficient signal to noise.

We start by using some known constants:

In [1]:
FE_ABS_CS_1 = 398.3 # cm^2/g
FE_ABS_CS_2 = 51.08 # cm^2/g 408.1 at edge
FE_AW = 55.845 # g/mol
NA = 6.022e23 # mol^-1
EV_TO_JOULES = 1.6e-19

Flow constants:

In [2]:
WATER_VISCOSITY = 8.90e-4 # Pa*s N*s/m^2 @ 25C Note @ 20C 1.0016E-3, @ 30C 7.9722E-4
SURFACE_TENSION = 0.07197 # N/m @ 25 C
WATER_DENSITY = 997.0 # kg/m^3 @ 25 C
WE_J = 0.8 # Weber number for jetting > 0.8

Some constants from the experiment:

In [3]:
SPOT_DIAMETER = 120 # nm
PHOTON_1 = 7186 # eV
PHOTON_2 = 7110 # eV
E_BW = 15 # eV
E_RES = 0.2 # eV
PULSE_DURATION = 10 # fs
TUBE_RADIUS = 1.27e-4 / 2.0 # m

Some parameters to control:

In [4]:
jet_thickness = 50 # um, between 5 um and 50 um
molar_con = 10 # mM (mol/m^3) (mmol/L)
pulse_energy_1 = 0.4 # uJ
pulse_energy_2 = 0.4 # uJ
loss_rate = 1

Now start doing some calculations:

In [5]:
import math
focal_area = math.pi * (SPOT_DIAMETER*1e-7 / 2)**2 # cm^2
focal_volume = jet_thickness*1e-4 * focal_area # cm^3

n_iron = NA * molar_con * focal_volume*1e-6
mass_iron = FE_AW / NA # g

n_photons_1 = pulse_energy_1*1e-6 / EV_TO_JOULES / PHOTON_1
n_photons_2 = pulse_energy_2*1e-6 / EV_TO_JOULES / PHOTON_2

Find the probability that an iron atom absorbs a photon from the first pulse:

In [6]:
abs_rate_1 = n_photons_1 * (FE_ABS_CS_1 * mass_iron) / focal_area
print('The absorption rate of the first pulse is {:.2f}%'.format(abs_rate_1*100))

The absorption rate of the first pulse is 11.36%


Now find the number of photons absorbed in the second pulse

In [7]:
abs_rate_2 = (n_photons_2 * FE_ABS_CS_2 * mass_iron) / focal_area
n_photons_abs = abs_rate_2 * n_iron
n_photons_abs_bin = n_photons_abs / (E_BW / E_RES)
print('The absorption rate of the second pulse is {:.2f}%'.format(abs_rate_2*100))

The absorption rate of the second pulse is 1.47%


In each energy bin, we expect to see changes of about 10% at our signals.
Let's err on the side of caution and expect a change of 1% out of the excitation rate of our molecules.
Compare this to the necessary number of photons in each bin to see this change in the signal.

In [8]:
nec_n_photons_bin = 1 / (abs_rate_2 * 0.01)**2
averaging_time = nec_n_photons_bin / n_photons_abs_bin / 120/60 # minutes
print('We will need to average for {:.1F} minutes'.format(averaging_time))
fluence_1 = pulse_energy_1 / focal_area / PULSE_DURATION # W/cm^2
fluence_2 = pulse_energy_2 / focal_area / PULSE_DURATION # W/cm^2
print('We will use a fluence of {:.1E} W/cm^2 for the first pulse and {:.1E} W/cm^2 for the second pulse'.format(fluence_1, fluence_2))

We will need to average for 9.6 minutes
We will use a fluence of 3.5E+08 W/cm^2 for the first pulse and 3.5E+08 W/cm^2 for the second pulse


Calculate the flow and necessary volume.

In [9]:
jet_flow = math.sqrt((WE_J * (math.pi**2) * (TUBE_RADIUS**3) * SURFACE_TENSION) / WATER_DENSITY) *1e6*60 # mL/minute
nec_volume = jet_flow * loss_rate *60*12*4*1e-3 # L
nec_mass = molar_con * nec_volume * FE_AW # mg
print('We will need at least {:.1F} liters of solution and {:.0F} milligrams of solute for four shifts'.format(nec_volume, nec_mass))

We will need at least 2.1 liters of solution and 1166 milligrams of solute for four shifts
